# Time Series Analysis: Midterm Exam

`MENE, MENE, TEKEL, UPHARSIN.
-The Book of Daniel, Chapter 5: Belshazzar's Feast`

---

## Libraries

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor
from pandas.plotting import register_matplotlib_converters
from IPython.display import display
from tsa_functions import *

register_matplotlib_converters()
sns.set_style('darkgrid')

np.set_printoptions(precision=2)
pd.set_option('precision', 2)

In [3]:
import json
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
import tqdm
from IPython.display import clear_output
from itertools import product
import warnings


warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [62]:
def mae(y_true, y_pred):
    score = np.mean(np.abs(y_true - y_pred))
    return score


def rmse(y_true, y_pred):
    if len(y_true) != len(y_pred):
        raise ValueError('Lengths Mismatch')
    score = np.sqrt(np.mean((y_true - y_pred)**2))
    return score


def mase(y_true, y_pred, s_ts):
    if len(y_true) != len(y_pred):
        raise ValueError('Lengths Mismatch')
    ts = s_ts.to_numpy()
    score = np.mean(
        np.abs((y_true - y_pred)/np.mean(np.abs(ts[1:] - ts[:-1]))))
    return score


def rmsse(y_true, y_pred, ts):
    if len(y_true) != len(y_pred):
        raise ValueError('Lengths Mismatch')
    ts = ts.to_numpy()
    score = np.sqrt(
        np.mean(((y_true - y_pred)/np.mean(np.abs(ts[1:] - ts[:-1])))**2))
    return score


def rateMyForecast(train, test, forecast):
    """
    Evalute the forcast per group, given train, test, and forecast tables.

    The function evaluates the metrics per column of the provided table.

    Parameters
    ----------
    train : DataFrame
        DataFrame contaning the train set.
    test : DataFrame
        DataFrame contaning the train set.
    forecast : DataFrame
        DataFrame contaning the train set.

    Returns
    -------
    DataFrame
        DataFrame contaning the metrics as columns, groups as rows,
        and scores as values.

    """
    res = pd.DataFrame([
        {'Group': col,
         'MAE': mae(test[col], forecast[col]),
         'RMSE': rmse(test[col], forecast[col])}
        for col in test])
    display(res.set_index('Group'))
    return res.set_index('Group')


def TimeseriesGenerator(X, y, w, h):
    X_train = np.array(*timeseries_dataset_from_array(
        X, targets=None, sequence_length=w, end_index=len(X)-h))
    y_train = np.array(*timeseries_dataset_from_array(
        X, targets=None, sequence_length=h, start_index=w))
    X_test = X[None, -w:]
    y_test = y[None, :]
    return X_train, X_test, y_train, y_test


def cross_val_score(X, est, config, scoring, cv):
    param = config.copy()
    h = param.pop('h')
    w = param.pop('w')
    folds = cv.split(X, h)
    scores = {metric: [] for metric in scoring}
    for train, val in folds:
        X_train, X_test, y_train, y_test = TimeseriesGenerator(
            train, val, w, h)
        est.set_params(**param)
        est.fit(X_train, y_train)
        y_hat = est.predict(X_test)
        for metric in scores:
            scores[metric].append(scoring[metric](y_test, y_hat))
    return scores


def cross_val_predict(X, est, config, cv):
    param = config.copy()
    h = param.pop('h')
    w = param.pop('w', None)
    folds = cv.split(X, h)
    fit_params = {}
    res = {}
    for k, (train, val) in enumerate(folds):
        if w:
            X_train, X_test, y_train, y_test = TimeseriesGenerator(
                train, val, w, h)
            est.set_params(**param)
            est.fit(X_train, y_train)
            y_hat = est.predict(X_test)[0]
        else:
            try:
                model = est(X, **param)
                fit = model.fit(**fit_params)
                y_hat = fit.forecast(h)
            except:
                y_hat = np.full(len(val), np.nan)
        res[k] = y_hat
    return res

class TimeSeriesSplit:
    def __init__(self, val_size):
        self.val_size = val_size

    def split(self, design_set, h):
        val_end = len(design_set)
        divider = val_end - h
        dataset = []
        while len(design_set) - divider <= self.val_size:
            dataset.append(
                (design_set[np.arange(0, divider)],
                 design_set[np.arange(divider, val_end)]))
            val_end -= 1
            divider -= 1
        return dataset[::-1]


class GridSearchCV:
    def __init__(self, estimator, param_grid, cv, scoring):
        self.est = estimator
        self.param_grid = param_grid
        self.param_list = [
            dict(zip(param_grid.keys(), params))
            for params in product(*param_grid.values())]
        self.cv = cv
        self.scoring = scoring

    def fit(self, X, scores=False):
        self.cv_results_ = []
        self.df_records_ = []
        for param in tqdm.tqdm(self.param_list):
            if scores:
                res = {
                    'params': param.copy(),
                    **cross_val_score(
                        X, self.est, param, self.scoring, self.cv)}
                rec = {
                    'Lookback': res['params']['w'],
                    'Horizon': res['params']['h'],
                    'Average RMSE': np.mean(res['rmse']),
                    'Stdev RMSE': np.std(res['rmse'])}
                rec['Sum'] = (rec['Average RMSE'] + rec['Stdev RMSE'])
#                 self.best_params = (
#                     self.df.nsmallest(1, 'Sum').iloc[0].to_dict())
            else:
                res = {
                    'params': param.copy(),
                    **cross_val_predict(
                        X, self.est, param, self.cv)}
                rec = res
            self.cv_results_.append(res)
            self.df_records_.append(rec)
        self.df = pd.DataFrame(self.df_records_)
                

def forecastUsingConfig(est, regions, design_set, test_set):
    forecast = {}
    for region in regions:
        train = design_set[region['Region']]
        test = test_set[region['Region']]
        w = int(region['Lookback'])
        h = int(region['Horizon'])
        X_train, X_test, y_train, y_test = TimeseriesGenerator(
            train, test, w, h)
#         est.set_params(**param)
        fit = est.fit(X_train, y_train)
        forecast[region['Region']] = fit.predict(X_test)[0]
    forecast_set = pd.DataFrame(forecast)
    forecast_set.index = test_set.index
    return forecast_set

---

## Forecasting Australian Domestic Tourism using Machine Learning 

Suppose that you've been hired by the Australian government to forecast the quarterly domestic tourism demand for the entire country.

Your goal is to generate forecasts using `LightGBM` models.

---

Load the Australian domestic tourism dataset and perform the following:

- Recode the `State` variable, similar to what we did in the discussion notebook.

- Create a hierarchical time series with overnight trips in regions at the bottom level of the hierarchy (which can then be aggregated to states, then aggregated to the national `Total`), similar to what we did in the previous notebooks.

- Re-index the series using `pd.period_range` instead of the strings in `Quarter`.

- Withhold the last two years (8 quarters) as a test set. 

- Withhold the last four years (16 quarters) in the training set as a validation set.

In [14]:
state_dict = {
    'New South Wales': 'NSW',
    'Northern Territory': 'NT',
    'Queensland': 'QLD',
    'South Australia': 'SA',
    'Tasmania': 'TAS',
    'Victoria': 'VIC',
    'Western Australia': 'WA'}
df = pd.read_csv('special_datasets/tourism.csv')
df.State.replace(state_dict, inplace=True)

df.index = pd.DatetimeIndex(df.Quarter).to_period('Q')
# df['Year'] = pd.to_datetime(df.Quarter).dt.year
# df['Quarter'] = pd.to_datetime(df.Quarter).dt.quarter

df_pivot = df.pivot_table(
    values='Trips',
    index=df.index,
    columns=['State', 'Region'],
    aggfunc=np.sum)

df_design = df_pivot.loc[: '2015Q4']
df_train = df_pivot.loc[: '2013Q4']
df_val = df_pivot.loc['2012Q1': '2015Q4']
df_test = df_pivot.loc['2016Q1':]
display(df_train, df_val, df_test)

State        ACT            NSW                                            \
Region  Canberra Blue Mountains Capital Country Central Coast Central NSW   
Quarter                                                                     
1998Q1    551.00         195.54          261.81        455.17      424.40   
1998Q2    416.03         200.41          238.60        343.48      557.01   
1998Q3    436.03         253.36          184.42        303.67      555.53   
1998Q4    449.80         245.57          229.09        331.82      590.16   
1999Q1    378.57         290.48          252.28        466.82      480.13   
1999Q2    558.18         294.80          212.37        312.84      459.64   
1999Q3    448.90         247.57          229.11        279.45      469.37   
1999Q4    594.83         241.55          254.27        363.10      564.14   
2000Q1    599.67         196.97          225.96        371.46      413.60   
2000Q2    557.14         233.54          321.48        335.44      467.09   
2000Q3    440.01         281.89          267.70        283.40      449.40   
2000Q4    494.38         158.35          245.76        372.94      473.35   
2001Q1    390.56         180.08          164.82        394.21      415.55   
2001Q2    618.38         167.62          265.28        328.43      410.98   
2001Q3    547.03         199.19          243.04        273.75      524.12   
2001Q4    532.92         150.39          286.65        406.40      481.11   
2002Q1    448.50         177.91          169.11        441.42      437.95   
2002Q2    547.35         249.95          257.06        311.96      592.84   
2002Q3    456.07         206.68          245.79        307.40      548.40   
2002Q4    521.87         174.17          249.99        425.08      534.61   
2003Q1    378.34         127.92          192.04        411.93      448.76   
2003Q2    445.89         144.79          274.27        347.68      542.87   
2003Q3    540.21         159.10          255.38        390.39      586.37   
2003Q4    580.24         158.26          237.92        332.69      574.22   
2004Q1    492.14         137.60          218.85        508.41      387.14   
2004Q2    483.45         168.14          240.95        384.30      534.49   
2004Q3    423.90         109.35          213.31        279.65      480.28   
2004Q4    481.61         123.74          238.77        340.19      515.91   
2005Q1    517.94         142.78          191.07        417.69      493.22   
2005Q2    478.37         189.33          238.57        274.11      453.99   
2005Q3    567.77         164.21          151.02        204.38      443.35   
2005Q4    423.20         157.38          210.53        308.49      443.79   
2006Q1    447.95         138.51          187.94        489.68      436.51   
2006Q2    549.93         179.30          230.12        320.43      448.85   
2006Q3    454.97         163.92          192.93        272.36      454.00   
2006Q4    439.93         168.51          217.35        342.70      413.38   
2007Q1    495.26         140.10          180.55        418.00      405.03   
2007Q2    476.02         207.16          250.32        318.00      507.87   
2007Q3    522.70         172.17          210.48        248.73      438.37   
2007Q4    461.29         126.47          193.84        331.01      428.74   
2008Q1    474.24         150.45          237.94        452.94      468.80   
2008Q2    482.66         197.77          215.69        215.62      417.84   
2008Q3    529.50         191.18          236.12        241.41      441.51   
2008Q4    469.84         147.07          235.60        299.15      398.01   
2009Q1    492.51         129.56          184.18        381.84      343.22   
2009Q2    396.75         143.57          190.39        246.81      434.09   
2009Q3    410.16         169.61          191.00        248.20      370.93   
2009Q4    427.86         128.22          311.34        293.81      456.66   
2010Q1    616.81         142.83          224.28        370.73      356.77

State        ACT            NSW                                            \
Region  Canberra Blue Mountains Capital Country Central Coast Central NSW   
Quarter                                                                     
2012Q1    446.90         184.05          246.43        359.68      423.28   
2012Q2    456.80         259.42          294.35        248.54      526.63   
2012Q3    521.84         190.13          216.09        309.32      394.86   
2012Q4    511.77         131.05          287.32        340.72      505.68   
2013Q1    524.55         193.22          304.09        377.22      363.20   
2013Q2    475.53         181.79          205.41        346.69      560.40   
2013Q3    506.51         197.82          248.79        258.91      404.99   
2013Q4    529.58         161.39          301.83        319.05      441.18   
2014Q1    540.61         179.77          267.64        414.37      339.37   
2014Q2    586.83         264.54          242.88        296.20      472.07   
2014Q3    606.21         190.81          236.87        216.51      500.96   
2014Q4    526.94         202.23          252.65        312.22      535.27   
2015Q1    566.14         155.08          256.96        418.76      378.21   
2015Q2    516.87         312.24          215.33        244.05      530.00   
2015Q3    688.20         191.64          287.85        258.28      473.80   
2015Q4    597.25         226.24          345.69        339.23      549.65   

State                                                                        \
Region   Hunter New England North West North Coast NSW Outback NSW Riverina   
Quarter                                                                       
2012Q1   703.70                 337.47         1140.21       39.00   183.60   
2012Q2   601.29                 329.13          923.79       80.28   217.94   
2012Q3   656.98                 340.88          971.24       71.90   176.51   
2012Q4   795.41                 350.31         1199.53       62.81   191.47   
2013Q1   711.30                 314.91         1234.21       45.97   182.31   
2013Q2   680.03                 268.22          936.41      104.12   271.84   
2013Q3   697.92                 299.03         1011.66      106.65   205.55   
2013Q4   659.91                 288.01         1114.82       72.21   195.86   
2014Q1   801.25                 344.79         1301.43       47.14   217.15   
2014Q2   692.44                 291.22         1174.74       99.99   217.30   
2014Q3   686.85                 310.23         1021.47      147.42   236.51   
2014Q4   785.36                 337.74         1178.52      104.22   254.28   
2015Q1   838.81                 335.50         1306.61       68.17   253.15   
2015Q2   764.05                 379.58         1193.63      112.08   260.62   
2015Q3   699.69                 294.54         1097.94      126.27   264.26   
2015Q4   831.23                 300.35         1266.60      105.12   240.07   

State                                                              NT         \
Region  Snowy Mountains South Coast   Sydney The Murray Alice Springs Barkly   
Quarter                                                                        
2012Q1           134.11     1038.10  2050.56     188.97         38.70   8.31   
2012Q2           126.22      755.47  1934.29     226.84         43.22  12.11   
2012Q3           300.97      506.48  1923.70     202.57         65.68  17.31   
2012Q4           127.88      764.52  1919.27     227.13         25.67  11.93   
2013Q1           160.73     1057.57  1797.54     228.71         18.97   6.80   
2013Q2           129.25      696.38  2105.89     267.39         40.75  12.95   
2013Q3           294.91      678.77  2181.61     175.48         61.05  19.73   
2013Q4           111.12      823.92  2262.10     286.37         36.06   8.86   
2014Q1           138.31     1027.84  2025.43     216.93         33.33  10.59   
2014Q2           151.71      765.71  1976.86     240.09         83.35   5.

State        ACT            NSW                                            \
Region  Canberra Blue Mountains Capital Country Central Coast Central NSW   
Quarter                                                                     
2016Q1    625.14         227.46          306.44        398.09      508.69   
2016Q2    592.61         303.56          322.50        250.96      461.04   
2016Q3    572.44         211.02          262.93        265.45      505.24   
2016Q4    667.21         220.17          277.33        379.48      525.25   
2017Q1    634.37         190.47          300.02        455.06      529.94   
2017Q2    748.29         315.79          305.71        278.48      651.36   
2017Q3    631.76         314.63          287.25        272.44      628.60   
2017Q4    720.33         203.46          356.38        427.77      628.23   

State                                                                         \
Region    Hunter New England North West North Coast NSW Outback NSW Riverina   
Quarter                                                                        
2016Q1    842.80                 321.22         1560.82       65.68   238.92   
2016Q2    829.98                 392.02         1153.70      134.83   279.95   
2016Q3    801.06                 350.37         1038.75      141.24   254.54   
2016Q4    843.79                 346.69         1407.18      105.95   230.67   
2017Q1    757.03                 358.09         1409.74       91.14   182.64   
2017Q2    897.96                 405.91         1164.06      107.54   293.57   
2017Q3    914.82                 398.97         1145.65      162.59   213.90   
2017Q4   1004.33                 404.19         1329.11      106.31   218.14   

State                                                              NT         \
Region  Snowy Mountains South Coast   Sydney The Murray Alice Springs Barkly   
Quarter                                                                        
2016Q1           136.59     1210.63  2199.58     270.96         26.56  23.36   
2016Q2            98.22      775.57  2389.05     269.21         76.55  14.11   
2016Q3           398.11      653.47  2217.42     226.70        120.97  36.34   
2016Q4           143.47      923.95  2369.82     221.75         40.29  16.43   
2017Q1           159.37     1189.25  2426.30     271.65         48.00  17.94   
2017Q2           162.18      938.97  2446.16     317.35         85.13  44.52   
2017Q3           448.99      735.30  2515.94     259.18         80.81  21.26   
2017Q4           149.81      904.13  2537.20     273.43         52.25   7.47   

State                                                                 QLD  \
Region   Darwin Kakadu Arnhem Katherine Daly Lasseter MacDonnell Brisbane   
Quarter                                                                     
2016Q1   145.05         51.87          67.19    17.58      20.48  1327.24   
2016Q2   168.07         72.38          68.62    61.40       8.00  1434.34   
2016Q3   232.36        113.06          92.34    89.19      29.57  1496.62   
2016Q4   139.79         67.05          50.46    21.36       5.30  1745.86   
2017Q1   109.65         39.45          49.74    21.93      11.46  1550.14   
2017Q2   223.19         65.85         104.10    65.68      32.98  1601.92   
2017Q3   226.11         81.87          88.53    62.57      36.51  1786.03   
2017Q4   150.25         43.72          40.27    33.42      18.68  1540.97   

State                                                                       \
Region  Bundaberg Central Queensland Darling Downs Fraser Coast Gold Coast   
Quarter                                                                      
2016Q1      91.84             282.91        361.78       143.96     933.09   
2016Q2     133.99             338.91        525.31       168.44     854.11   
2016Q3     169.67             447.11        455.37       158.09     850.26   
2016Q4     139.27             382.08        464.81       117.11    1065.63

### Q1.

Given that:

- Our goal is to create an 8-step forecasting model.


- We have allocated 16 observations for the validation set. 

How many train-val splits will we end up with under TSCV?

In [15]:
a = np.zeros((9, 16))
for i, arr in enumerate(a):
    arr[0:i+8] = 1
    arr[:i] = -1
df_0 = pd.DataFrame(a.astype(int))
df_0.columns = range(-16, 0)
df_0.index = range(1, 10)
df_0.style.background_gradient(cmap='coolwarm', axis=None)

,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1
1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0
2,-1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0
3,-1,-1,1,1,1,1,1,1,1,1,0,0,0,0,0,0
4,-1,-1,-1,1,1,1,1,1,1,1,1,0,0,0,0,0
5,-1,-1,-1,-1,1,1,1,1,1,1,1,1,0,0,0,0
6,-1,-1,-1,-1,-1,1,1,1,1,1,1,1,1,0,0,0
7,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,1,1,0,0
8,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,1,1,0
9,-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,1,1


### Answer.

`9 train-val splits`

### Q2.

- Extract the `Canberra` time series.


- Using TSCV on the validation set, calculate the average and standard deviation of the `RMSE` statistic for an "out-of-the-box" `LightGBM` model (i.e. default parameters) that uses a `direct forecasting` approach.


- Set the `random_state` parameter of the `LGBMRegressor()` to 1.


- Search the range of `lookback windows` from <b><u>4 to 16</u></b> and find the optimal lookback that minimizes the sum of the average and stdev. `RMSE` values.


- Place the statistics in a dataframe and print it.

<i>HINT: I suggest you try to implement this from scratch. If you copy paste code without understanding what you're doing, you will most probably get this wrong. It is critical that you get this question right so that bugs/errors will not cascade further down.</i>

In [98]:
model = 'MOR-LGB'
component = 'trend'
ts = df_design[col]
name = 'AAA'
path = r'C:/Users/aamorado/Moog Inc/The Journey towards a Smart Connected Factory - MOOG - AIM/PM/mazak1011/time_series_simulation'
dt = datetime.now().strftime("%d-%m-%y %H-%M-%S")
h = 8
    
est = MultiOutputRegressor(lgb.LGBMRegressor(random_state=1), n_jobs=-1)
param_grid = {
    'w': range(4, 17),
    'h': [h]
}
tscv = TimeSeriesSplit(val_size=16)
gs = GridSearchCV(est, param_grid, tscv, {'rmse': rmse})

col = ('ACT', 'Canberra')

gs.fit(ts)

100%|██████████| 13/13 [00:20<00:00,  1.55s/it]


In [99]:
import pickle
obj = (
    gs
    .df
    .assign(
        model=model, component=component,
        params=lambda x: x.params.astype(str))
    .set_index(['component', 'model', 'params'])
)
display(obj)
with open(f'{path}/{name} {dt}.pickle', 'wb') as handle:
    pickle.dump(
        obj,
        handle)

0  \
component model   params                                                                                                                                                                             
trend     MOR-LGB {'w': 4, 'h': 8}     [521.7200607630713, 488.24357110322256, 498.4733511829438, 499.49545948483814, 499.403377003239, 452.7435634974129, 501.15914153158366, 472.01830199364855]   
                  {'w': 5, 'h': 8}    [533.196462258115, 478.37158535477386, 510.94483533654625, 475.8105499531224, 498.02902072437547, 448.12214957453534, 508.84188055797813, 485.3194656999158]   
                  {'w': 6, 'h': 8}     [528.2062489006604, 489.69437621162535, 501.5612763520464, 467.24768292407674, 478.0741463792554, 454.88120162557885, 522.1957131915813, 494.2726088319394]   
                  {'w': 7, 'h': 8}      [505.5139479817778, 499.5974921321233, 480.83183535536136, 475.92478748159954, 488.8114997451794, 445.962007517106, 518.9099448001282, 482.73930187599643]   
                  {'w': 8, 'h': 8}   [490.7295348900334, 478.6797595685413, 470.93187925505276, 480.36573146216176, 478.72983249085956, 445.4842172495295, 491.48531917107755, 473.87760837914504]   
                  {'w': 9, 'h': 8}     [487.89156875610354, 487.83115310668944, 487.92638931274416, 487.1996269226074, 486.9581748962402, 482.6873329162598, 481.3963310241699, 479.9619758605957]   
                  {'w': 10, 'h': 8}    [486.11609434470154, 489.0572235889924, 487.7609268579728, 489.6776044796675, 483.58834056365185, 481.0374982784956, 480.0751475798778, 480.76863372020233]   
                  {'w': 11, 'h': 8}     [487.32929671438114, 488.9171656558388, 490.3188854016756, 486.29065423262745, 481.9187999524568, 479.67210388183594, 480.9060114810341, 479.016526874743]   
                  {'w': 12, 'h': 8}    [487.1387527053421, 491.57550791147594, 486.8577278755807, 484.64902640677786, 480.5403219686972, 480.51453049118453, 479.1102633089633, 479.6365760597023]   
                  {'w': 13, 'h': 8}   [489.8215374416775, 488.05311330159503, 485.1862513224284, 483.30809699164496, 481.43026648627387, 478.65802595350476, 479.7501398722331, 478.4634187486437]   
                  {'w': 14, 'h': 8}       [486.1483895438058, 486.3680341448103, 483.822358921596, 484.302547781808, 479.5468828473772, 479.30326363699777, 478.5467084612165, 481.32420305524556]   
                  {'w': 15, 'h': 8}     [484.3577279483571, 484.9987855799058, 484.8611836152918, 482.44824936810664, 480.23724095961626, 478.0512937657973, 481.4940831801471, 482.3663033878102]   
                  {'w': 16, 'h': 8}        [482.88606863310844, 486.1047391024503, 482.96762269915956, 483.2474476207386, 478.97563495058, 481.0729703036222, 482.572910193241, 480.6134652802439]   

                                                                                                                                                                                                 1  \
component model   params                                                                                                                                                                             
trend     MOR-LGB {'w': 4, 'h': 8}    [495.13693516824213, 511.31372771216144, 498.81116822159385, 510.0526589530193, 463.3260332529855, 468.8441996096802, 473.9665092217352, 472.35300351241455]   
                  {'w': 5, 'h': 8}      [511.4972067685701, 499.7275656057032, 517.8860296597094, 494.93175246088293, 443.34111568512265, 470.2672332351938, 486.1773176891719, 495.6886036761936]   
                  {'w': 6, 'h': 8}       [496.6446976460287, 517.0871170090712, 498.67115469619057, 489.9310685431094, 429.2362041584299, 478.961968853472, 498.5950293202243, 502.22063892299656]   
                  {'w': 7, 'h': 8}      [499.3927950309144, 516.8507094161363, 495.8241370289659, 475.20537885229027, 439.5946860956242, 489.8304219644521, 499.64694490437915, 496.4760921173173]   
                  {'w': 8, 'h': 8}

In [9]:
gs.df.drop(columns=['Horizon'])

,Lookback,Average RMSE,Stdev RMSE,Sum
0,4,66.84,24.38,91.21
1,5,67.72,24.38,92.09
2,6,67.33,26.25,93.58
3,7,66.84,25.43,92.26
4,8,66.66,21.89,88.55
5,9,69.11,22.25,91.35
6,10,69.88,24.09,93.97
7,11,71.39,25.30,96.69
8,12,72.42,26.49,98.91
9,13,72.68,26.91,99.59


<div class="alert alert-block alert-info">
    The optimal lookback that minimizes the sum of the average and stdev. <code>RMSE</code> values.
</div>

In [8]:
pd.DataFrame(gs.best_params, index=[col]).drop(columns=['Horizon'])

,Lookback,Average RMSE,Stdev RMSE,Sum
"(ACT, Canberra)",8.0,66.66,21.89,88.55


### Q3.

- Repeat the process done in Q2 for <b><u>for all regions</u></b>.


- Only print out the final dataframe with the optimal lookbacks.

In [9]:
regions = []
for label, content in tqdm.tqdm(list(df_design.items())):
    gs.fit(content)
    regions.append({
        'Region': label,
        **gs.best_params
    })

with open('best_params.json', 'w') as fp:
    json.dump(regions, fp)

100%|██████████| 76/76 [18:54<00:00, 14.92s/it]


In [10]:
pd.DataFrame(regions).set_index('Region').drop(columns=['Horizon'])

,Lookback,Average RMSE,Stdev RMSE,Sum
Region,,,,
"(ACT, Canberra)",8.0,66.66,21.89,88.55
"(NSW, Blue Mountains)",4.0,40.43,9.73,50.16
"(NSW, Capital Country)",4.0,41.67,6.88,48.55
"(NSW, Central Coast)",7.0,38.12,5.50,43.61
"(NSW, Central NSW)",5.0,68.05,4.99,73.05
"(NSW, Hunter)",10.0,66.50,11.63,78.13
"(NSW, New England North West)",16.0,30.03,2.11,32.14
"(NSW, North Coast NSW)",11.0,129.40,11.69,141.10
"(NSW, Outback NSW)",8.0,25.37,1.60,26.98


### Q4.

- Retrain the LightGBM models on the full training set with their respective optimal lookbacks.


- Evaluate them on their test sets and compile the `MAE` and `RMSE` statistics in a dataframe then print them.

In [11]:
df_forecast = forecastUsingConfig(est, regions, df_design, df_test)

In [12]:
rms_region = rateMyForecast(
    df_design,
    df_test,
    df_forecast)

,MAE,RMSE
Group,,
"(ACT, Canberra)",140.75,154.26
"(NSW, Blue Mountains)",60.80,75.68
"(NSW, Capital Country)",54.24,63.71
"(NSW, Central Coast)",31.61,46.02
"(NSW, Central NSW)",87.58,103.34
"(NSW, Hunter)",195.06,207.98
"(NSW, New England North West)",52.92,58.10
"(NSW, North Coast NSW)",229.33,252.25
"(NSW, Outback NSW)",21.75,26.85


### Q5.

- Produce aggregate forecasts for the `State` and `Total` levels using the regional forecasts (i.e. bottom-up aggregation).


- Calculate the `MAE` and `RMSE` statistics and print them as a dataframe.

In [13]:
rms_state = rateMyForecast(
    df_train.sum(level=0, axis=1),
    df_test.sum(level=0, axis=1),
    df_forecast.sum(level=0, axis=1))
df_forecast_total = df_forecast.sum(axis=1).rename('Total').to_frame()
rms_total = rateMyForecast(
    df_train.sum(axis=1).rename('Total').to_frame(),
    df_test.sum(axis=1).rename('Total').to_frame(),
    df_forecast_total)
clear_output()
pd.concat([rms_total, rms_state])

,MAE,RMSE
Group,,
Total,4608.40,4687.60
ACT,140.75,154.26
NSW,1220.04,1272.86
NT,143.60,173.27
QLD,836.33,884.42
SA,260.83,278.45
TAS,132.88,151.25
VIC,1230.51,1294.25
WA,653.45,674.67


### Q6.

- Calculate the average `MAE` and `RMSE` across all time series (i.e. all regions, states, and the total series).

In [14]:
(pd
 .concat([rms_region, rms_state, rms_total])
 .mean()
 .rename('All Series')
 .to_frame()
 .T
 )

,MAE,RMSE
All Series,167.11,179.87


### Q7.

- Compare the forecasting performance of LightGBM to the ETS/ARIMA models in Assignment 3.


- Which performed better? Why is this the case?


### Answer.

<div class="alert alert-block alert-info">
    <table>
        <tr>
            <td></td>
            <td><b>MAE</b></td>
            <td><b>RMSE</b></td>
        </tr>
        <tr>
            <td>ETS/ARIMA</td>
            <td>83.58</td>
            <td>97.20</td>
        </tr>
        <tr>
            <td>LightGBM</td>
            <td>167.11</td>
            <td>179.87</td>
        </tr>
    </table>
    <ul>
        <li><code>ETS</code>/<code>ARIMA</code> implementation has been more rigorously tuned than the <code>LightGBM</code> implementation here.</li> In the assignment I did at total of ($104+144$) models per time series but here we only did $12$.
        <li>ML models such as GBM are slow to converge and needs a lot of data. For econometric data such as this, <code>statsmodels</code> should be better.</li>
    </ul>
</div>

### Q8.

- What are some things we can do to improve forecast accuracy?

### Answer.

<div class="alert alert-block alert-info">
    <ol>
        <li>If we are sticking to <code>LightGBM</code>, here are our options:</li>
        <ul>
            <li>We can tune <code>LightGBM</code>'s hyper parameters. There are A LOT ($\gg10$). These ought to increase accuracy, but we should avoid overfitting.</li>
            <li>Use <code>sklearn.multioutput.RegressorChain</code> wrapper instead of the <code>MultioutputRegressor</code>.  <code>RegressorChain</code> chains the outputs of the model as features of the next step when outputing multi-step forecasts. This makes sense as there is reason to believe that the previous days' output has correlation with the next.</li>
        </ul><br>
        <li>If we are concerned with the dataset:</li>
        <ul>
            <li><code>LightGBM</code> is capable of using exogenous variables and can increase accuracy of data is available.</li>
            <li>More data would be appreciated.</li>
        </ul><br>
        <li>Other things that can make the <strike>accuracy go brrr</strike> metrics better:</li>
        <ul>
            <li>Try other ML models like <code>XGBoost</code>, <code>CatBoost</code>, ensemble methods.</li>
            <li>Given enough data, we can do <code>RNN</code> based approaches such <code>LSTM</code> and <code>GRU</code>, and maybe even <code>transformers</code>.</li>
        </ul>
    </ol>
</div>